In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

import matplotlib
import matplotlib.image as img
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
pd.set_option("future.no_silent_downcasting", True)

In [4]:
train = pd.read_csv('data_housing_model/train.csv')

In [5]:
train.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


# Обработка данных

In [6]:
train = train.rename(columns={'LifeSquare': 'Life_Square', 'KitchenSquare': 'Kitchen_Square', 'HouseFloor': 'House_Floor', 'HouseYear': 'House_Year'})

### Меняем кол-во комнат, равных 0, в зависимости от площади

In [7]:
train[train['Rooms'] == 0]

,Id,DistrictId,Rooms,Square,Life_Square,Kitchen_Square,Floor,House_Floor,House_Year,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
1397,12638,27,0.0,138.427694,136.215499,0.0,4,3.0,2016,0.075424,B,B,11,3097,0,NaN,0,0,B,268394.744389
1981,7917,27,0.0,212.932361,211.231125,0.0,2,3.0,2008,0.211401,B,B,9,1892,0,NaN,0,1,B,302211.260887
2269,7317,27,0.0,41.790881,NaN,0.0,13,0.0,1977,0.211401,B,B,9,1892,0,NaN,0,1,B,98129.976788
3911,770,28,0.0,49.483501,NaN,0.0,16,0.0,2015,0.118537,B,B,30,6207,1,1183.0,1,0,B,217009.338463
4366,456,6,0.0,81.491446,NaN,0.0,4,0.0,1977,0.243205,B,B,5,1564,0,540.0,0,0,B,212864.799112
4853,3224,27,0.0,2.377248,0.873147,0.0,1,0.0,1977,0.017647,B,B,2,469,0,NaN,0,0,B,126596.941798
6149,3159,88,0.0,38.697117,19.345131,9.0,9,16.0,1982,0.127376,B,B,43,8429,3,NaN,3,9,B,158998.110646
8834,9443,27,0.0,87.762616,85.125471,0.0,5,15.0,1977,0.211401,B,B,9,1892,0,NaN,0,1,B,219281.918007


In [8]:
train.loc[(train.Square > 130) & (train.Rooms == 0), 'Rooms'] = 3
train.loc[((train.Square > 50) & (train.Square < 90)) & (train.Rooms == 0), 'Rooms'] = 2
train.loc[(train.Square < 50) & (train.Rooms == 0), 'Rooms'] = 1

In [9]:
train.groupby('Rooms', as_index=False).agg({'Id': 'count'}).rename(columns={'Id': 'count_of_rooms'})

,Rooms,count_of_rooms
0,1.0,3709
1,2.0,3882
2,3.0,2237
3,4.0,150
4,5.0,18
5,6.0,1
6,10.0,2
7,19.0,1


### Меняем количество комнат в соответствии с площадью

In [10]:
train.groupby('Rooms', as_index=False).agg({'Square': 'mean'}).rename(columns={'Square': 'mean_square'})

,Rooms,mean_square
0,1.0,41.314395
1,2.0,56.802556
2,3.0,76.991546
3,4.0,98.377544
4,5.0,122.614941
5,6.0,59.414334
6,10.0,59.964120
7,19.0,42.006046


In [11]:
train.loc[(train.Rooms == 6), 'Rooms'] = 2
train.loc[(train.Rooms == 10), 'Rooms'] = 2
train.loc[(train.Rooms == 19), 'Rooms'] = 1

### Удаляем две колонки из-за большого количества пропущенных значений

In [12]:
train.drop(['Life_Square', 'Kitchen_Square'], axis=1, inplace=True)

### Меняем площадь в зависимости от кол-ва комнат (попробовать удалить)

In [13]:
train.groupby('Rooms', as_index=False).agg({'Square': 'mean'}).rename(columns={'Square': 'mean_square'})

,Rooms,mean_square
0,1.0,41.314582
1,2.0,56.804856
2,3.0,76.991546
3,4.0,98.377544
4,5.0,122.614941


In [14]:
train.loc[(train.Square < 13) & (train.Rooms == 1), 'Square'] = 41.314582
train.loc[(train.Square < 13) & (train.Rooms == 2), 'Square'] = 56.804856
train.loc[(train.Square < 13) & (train.Rooms == 3), 'Square'] = 76.991546

### Меняем кол-во этажей в доме

In [15]:
train.loc[(train.House_Floor < train.Floor), 'House_Floor'] = train.Floor

### Меняем год постройки

In [16]:
train.loc[(train.House_Year == 4968), 'House_Year'] = 1968

In [17]:
train.loc[(train.House_Year == 20052011), 'House_Year'] = 2005

### Заменяем буквенное представление категориальных переменных на численное

In [18]:
train.replace({'A': 0, 'B': 1}, inplace=True)

### Добавляем новые колонки на основе существующих

In [19]:
train['house_age'] = 2024-train['House_Year']

In [20]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,10000.0,8383.407700,4859.019020,0.000000,4169.500000,8394.500000,12592.500000,16798.000000
DistrictId,10000.0,50.400800,43.587592,0.000000,20.000000,36.000000,75.000000,209.000000
Rooms,10000.0,1.888100,0.811446,1.000000,1.000000,2.000000,2.000000,5.000000
Square,10000.0,56.369596,20.986444,13.784865,41.782323,52.535436,65.912854,641.065193
Floor,10000.0,8.526700,5.241148,1.000000,4.000000,7.000000,12.000000,42.000000
House_Floor,10000.0,13.435700,6.094012,1.000000,9.000000,14.000000,17.000000,117.000000
House_Year,10000.0,1984.865700,18.411517,1910.000000,1974.000000,1977.000000,2001.000000,2020.000000
Ecology_1,10000.0,0.118858,0.119025,0.000000,0.017647,0.075424,0.195781,0.521867
Social_1,10000.0,24.687000,17.532614,0.000000,6.000000,25.000000,36.000000,74.000000
Social_2,10000.0,5352.157400,4006.799803,168.000000,1564.000000,5285.000000,7227.000000,19083.000000


In [21]:
train[['Floor', 'Social_1', 'Social_2', 'Social_3', 'Shops_1', 'house_age']] = train[['Floor', 'Social_1', 'Social_2', 'Social_3', 'Shops_1', 'house_age']].astype('float64')

In [22]:
train.set_index('Id', inplace=True)

In [23]:
train.drop('House_Year', axis=1, inplace=True)

In [24]:
train.drop('Healthcare_1', axis=1, inplace=True)

In [25]:
train.head()

,DistrictId,Rooms,Square,Floor,House_Floor,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price,house_age
Id,,,,,,,,,,,,,,,,
14038,35,2.0,47.981561,7.0,9.0,0.089040,1,1,33.0,7976.0,5.0,0,11.0,1,184966.930730,55.0
15053,41,3.0,65.683640,7.0,9.0,0.000070,1,1,46.0,10309.0,1.0,1,16.0,1,300009.450063,46.0
4765,53,2.0,44.947953,8.0,12.0,0.049637,1,1,34.0,7759.0,0.0,1,3.0,1,220925.908524,56.0
5809,58,2.0,53.352981,8.0,17.0,0.437885,1,1,23.0,5735.0,3.0,0,5.0,1,175616.227217,47.0
10783,99,1.0,39.649192,11.0,12.0,0.012339,1,1,35.0,5776.0,1.0,2,4.0,1,150226.531644,48.0


In [26]:
y = train['Price']
y.head()

Id
14038    184966.930730
15053    300009.450063
4765     220925.908524
5809     175616.227217
10783    150226.531644
Name: Price, dtype: float64

In [27]:
train.drop('Price', axis=1, inplace=True)

In [28]:
test = pd.read_csv('data_housing_model/test.csv')
test.rename(columns={'HouseFloor': 'House_Floor'}, inplace=True)

In [29]:
test.set_index('Id', inplace=True)
test.drop(['LifeSquare', 'KitchenSquare'], axis=1, inplace=True)
test.replace({'A': 0, 'B': 1}, inplace=True)
test['house_age'] = 2024-test['HouseYear']
test.drop('HouseYear', axis=1, inplace=True)
test.drop('Healthcare_1', axis=1, inplace=True)

In [30]:
test.head()

,DistrictId,Rooms,Square,Floor,House_Floor,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,house_age
Id,,,,,,,,,,,,,,,
725,58,2.0,49.882643,6,14.0,0.310199,1,1,11,2748,1,0,0,1,52
15856,74,2.0,69.263183,6,1.0,0.075779,1,1,6,1437,3,0,2,1,47
5480,190,1.0,13.597819,2,5.0,0.000000,1,1,30,7538,87,5,5,1,115
15664,47,2.0,73.046609,22,22.0,0.101872,1,1,23,4583,3,3,3,1,17
14275,27,1.0,47.527111,17,17.0,0.072158,1,1,2,629,1,0,0,0,7


# Стандартизация данных

In [31]:
train.dtypes

DistrictId       int64
Rooms          float64
Square         float64
Floor          float64
House_Floor    float64
Ecology_1      float64
Ecology_2       object
Ecology_3       object
Social_1       float64
Social_2       float64
Social_3       float64
Helthcare_2      int64
Shops_1        float64
Shops_2         object
house_age      float64
dtype: object

In [32]:
feature_num_names = [
    'Rooms',
    'Square',
    'Floor',
    'House_Floor',
    'Ecology_1',
    'Social_1',
    'Social_2',
    'Social_3',
    'Shops_1',
    'house_age'
]

In [33]:
another_col = [
    'DistrictId',
    'Ecology_2',
    'Ecology_3',
    'Helthcare_2',
    'Shops_2'
]

In [34]:
scaler = StandardScaler()

In [35]:
train_standardized = pd.DataFrame(scaler.fit_transform(train[feature_num_names]),
                              columns=feature_num_names, index=train.index)

In [36]:
test_standardized = pd.DataFrame(scaler.transform(test[feature_num_names]),
                              columns=feature_num_names, index=test.index)

In [37]:
scaler2 = MinMaxScaler()
train_normalized = pd.DataFrame(scaler2.fit_transform(train[feature_num_names]), columns=feature_num_names,
                               index=train.index)

In [38]:
test_normalized = pd.DataFrame(scaler2.transform(test[feature_num_names]), columns=feature_num_names,
                               index=test.index)

In [39]:
scaler3 = RobustScaler()
train_scaled = pd.DataFrame(scaler3.fit_transform(train.loc[:, train.columns]), 
                            columns=train.columns, index=train.index)

In [40]:
test_scaled = pd.DataFrame(scaler3.transform(test), columns=test.columns, index=test.index)

In [41]:
train_standardized = pd.concat([train[another_col], train_standardized], axis=1)

In [42]:
test_standardized = pd.concat([test[another_col], test_standardized], axis=1)

In [43]:
train_normalized = pd.concat([train[another_col], train_normalized], axis=1)

In [44]:
test_normalized = pd.concat([test[another_col], test_normalized], axis=1)

In [45]:
train.to_csv('data_housing_model/train_housing_prepared')

In [46]:
test.to_csv('data_housing_model/test_housing_prepared')

In [47]:
train_standardized.to_csv('data_housing_model/train_standardized_housing_prepared')

In [48]:
test_standardized.to_csv('data_housing_model/test_standardized_housing_prepared')

In [49]:
train_normalized.to_csv('data_housing_model/train_normalized_housing_prepared')

In [50]:
test_normalized.to_csv('data_housing_model/test_normalized_housing_prepared')

In [51]:
train_scaled.to_csv('data_housing_model/train_scaled_housing_prepared')

In [52]:
test_scaled.to_csv('data_housing_model/test_scaled_housing_prepared')

In [53]:
y = pd.DataFrame(y)

In [54]:
y.to_csv('data_housing_model/housing_y')